In [35]:
import pandas as pd
import numpy as np
import ast
import string
import collections
from nltk import WordNetLemmatizer
import nltk

movies = pd.read_csv("movie_data.csv")

movies['genres'] = pd.Series(ast.literal_eval(genres) for genres in movies['genres'])

movies.head(5)

,id,title,release_date,box_office_revenue,runtime,genres,summary
0,0,Ghosts of Mars,2001-08-24,14010832.0,98.0,"[Space western, Horror, Supernatural, Thriller...","Set in the second half of the 22nd century, th..."
1,1,White Of The Eye,1987,NaN,110.0,"[Erotic thriller, Psychological thriller, Thri...",A series of murders of rich young women throug...
2,2,A Woman in Flames,1983,NaN,106.0,[Drama],"Eva, an upper class housewife, becomes frustra..."
3,3,The Sorcerer's Apprentice,2002,NaN,86.0,"[Adventure, Fantasy, World cinema, Family Film]","Every hundred years, the evil Morgana returns..."
4,4,Little city,1997-04-04,NaN,93.0,"[Romance Film, Ensemble Film, Comedy-drama, Co...","Adam, a San Francisco-based artist who works a..."


In [2]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [44]:
#Helper funciton to map part-of-speech tags to something that our root-finding map "wordnet" can recognize
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
#Used to associate lemmatized tag object to literal string name
def tag_helper(word,tag):
    wntag = get_wordnet_pos(tag)
    if wntag is None:# not supply tag in case of None
        lemma = wnl.lemmatize(word)
        tb_tag = ""
    #assigning a variable to the english equivalent of our tags to be used later
    else:
        lemma = wnl.lemmatize(word, pos = wntag)
        if wntag == wordnet.ADJ:
            tb_tag = "Adjective"
        elif wntag == wordnet.VERB:
            tb_tag = "Verb"
        elif wntag == wordnet.NOUN:
            tb_tag = "Noun"
        elif wntag == wordnet.ADV:
            tb_tag = "Adverb"
        elif wntag is None:
            tb_tag = ""
    return lemma, tb_tag

In [59]:
wnl = WordNetLemmatizer()
wnl.lemmatize("eggs", "n")
#nltk.pos_tag(["eggs"])
test = nltk.tag.pos_tag("This is a test sentence The cat ran. We then walked home.".split())
test
wnl.lemmatize("mans")
#nltk.tag.pos_tag('a sentence'.split())

'man'

In [8]:
def tokenizer(blob):
    translator = str.maketrans({key: None for key in string.punctuation})
    
    wnl = WordNetLemmatizer()    
    tokenized = 
    
    tokenized = blob.lower().replace("-"," ").translate(translator).split()
    return tokenized

In [9]:
unique_genres = ["Drama", "Comedy", "Action/Adventure", "Romance Film", "Thriller"]

drama_list = []
comedy_list = []
action_list = []
romance_list = []
thriller_list = []

for genre in unique_genres:
    for genres, summary in zip(movies['genres'], movies['summary']):
        if genre in genres:
#             if genre == "Drama":
#                 drama_list.append(tb(summary))
#             elif genre == "Comedy":
#                 comedy_list.append(tb(summary))
#             elif genre == "Action/Adventure":
#                 action_list.append(tb(summary))
#             elif genre == "Romance Film":
#                 thriller_list.append(tb(summary))
#             elif genre == "Thriller":
#                 thriller_list.append(tb(summary))
            if genre == "Drama":
                drama_list.append(tokenizer(summary))
            elif genre == "Comedy":
                comedy_list.append(tokenizer(summary))
            elif genre == "Action/Adventure":
                action_list.append(tokenizer(summary))
            elif genre == "Romance Film":
                thriller_list.append(tokenizer(summary))
            elif genre == "Thriller":
                thriller_list.append(tokenizer(summary))
            

In [21]:
# vocab = set()
# for key in tf_dict.keys():
#     vocab.add(key[1])
# vocab = list(vocab)

tf_dict = {}

for n, summary in enumerate(drama_list):
    for word in summary:
        if (n, word) not in tf_dict:
            occurrence = summary.count(word)
            tf_dict[(n, word)] = occurrence

#This is just document frequency. Still need to implement inverse and log...
df_dict = {}
for key in tf_dict.keys():
    if key[1] in df_dict:
        df_dict[key[1]] += 1
    else:
        df_dict[key[1]] = 1

In [22]:
num_dramas = len(drama_list)

idf_dict = {}

for word, df in df_dict.items():
    idf_dict[word] = np.log(num_dramas / df)

In [27]:
sorted(idf_dict, key=idf_dict.get, reverse=True)[:150]

['naziism',
 'najaf',
 'gallifrey',
 'snehmoy',
 'surrogacy',
 'shiban',
 'belansais',
 'wetlands',
 'springsebert',
 'morros',
 'arbutny',
 'reugen',
 'mélièss',
 'lulled',
 'motherthe',
 'brightmores',
 'muthassi',
 'perpetratortête',
 'inclining',
 'topkapı',
 'christ’s',
 'arressted',
 'stagefright',
 'àlex’s',
 'nemerov',
 'khetarpal',
 'transvestism',
 'nationsmdashand',
 'armyref',
 'ballantyne',
 'janiss',
 'witwer',
 'amiri',
 'bruck',
 '1704',
 'spanned',
 '1740s',
 'marylou',
 '1685',
 'largeman',
 'satithe',
 'overload—she',
 'thapaal',
 'frochards',
 'reptilian',
 'talesmartin',
 'inflected',
 'gusciora',
 'pasto',
 'nadiya',
 'unsympathetically',
 'radars',
 'mrityudaata',
 'tomakawk',
 'mininente',
 'shanmmugams',
 'issue”',
 'kaate',
 'tagger',
 'basa',
 'rausch',
 'rabhani',
 'harleigh',
 'cioffi',
 'earthing',
 'plotdatecontinuity',
 'caanan',
 'restano',
 'curandera',
 'lastlisa',
 'ledru',
 'dog”',
 'eiji',
 'reaffirmed',
 'shais',
 'prisonfact',
 'videochapter39',


In [7]:
['this', 'is', 'a', 'sentence', 'sentence'].count('sentence')

2

## Here was my second attempt
Yet again we keep getting memory errors....

What words are characteristic of the movie summaries in those genres? Specifically:

- Drama
- Comedy
- Action/Adventure
- Romance film
- Thriller

In [2]:
unique_genres = ["Drama", "Comedy", "Action/Adventure", "Romance Film", "Thriller"]

def tokenizer(blob):
    translator = str.maketrans({key: None for key in string.punctuation})
    
    tokenized = blob.lower().replace("-"," ").translate(translator).split()
    return tokenized

In [3]:
movie_dict = {}
for genre in unique_genres:
    for title, genres, summary in zip(movies['title'], movies['genres'], movies['summary']):
        if genre in genres:
            movie_dict[(title, genre)] = tokenizer(summary)
            
word_index = {word: n for summary in movie_dict.values() for n, word in enumerate(summary)}
word_index;

In [4]:
# movie_vecs = {(value[0], value[1]): np.zeros(len(word_index)) for value in movie_dict.keys()}
# movie_vecs
movie_vecs = {}
for title, genre in movie_dict.keys():
    movie_vecs[(title, genre)] = np.zeros(len(word_index))

MemoryError: 

## The code below is our first attmempt

In [2]:
#all_genres stores all the genres split out of the lists and stacks horizontally
#This makes is easy to construct our dataframe later
all_genres = np.hstack(movies.genres)

#Each feature is stacked so that a value repeats for as many genres as were in the original genre list
all_titles = np.hstack([[title]*len(genre) for title, genre in movies[['title', 'genres']].values])
all_release_dates = np.hstack([[release]*len(genre) for release, genre in movies[['release_date', 'genres']].values])
all_revenues = np.hstack([[revenue]*len(genre) for revenue, genre in movies[['box_office_revenue', 'genres']].values])
all_runtimes = np.hstack([[runtime]*len(genre) for runtime, genre in movies[['runtime', 'genres']].values])
all_summaries = np.hstack([[summary]*len(genre) for summary, genre in movies[['summary', 'genres']].values])

In [2]:
movies_split = pd.DataFrame({'genres': np.hstack(movies.genres), 
                             'titles':np.hstack([[title]*len(genre) for title, genre in movies[['title', 'genres']].values]), 
                             'release_date':np.hstack([[release]*len(genre) for release, genre in movies[['release_date', 'genres']].values]),
                             'box_office_revenue':np.hstack([[revenue]*len(genre) for revenue, genre in movies[['box_office_revenue', 'genres']].values]), 
                             'runtime':np.hstack([[runtime]*len(genre) for runtime, genre in movies[['runtime', 'genres']].values]), 
                             'summary':np.hstack([[summary]*len(genre) for summary, genre in movies[['summary', 'genres']].values])})

In [3]:
movies_split = pd.DataFrame({'genres':all_genres, 'titles':all_titles, 'release_date':all_release_dates,
                     'box_office_revenue':all_revenues, 'runtime':all_runtimes, 'summary':all_summaries})

#Let's check to see if this yields what we expect...
movies_split.head(10)

,box_office_revenue,genres,release_date,runtime,summary,titles
0,14010832.0,Space western,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
1,14010832.0,Horror,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
2,14010832.0,Supernatural,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
3,14010832.0,Thriller,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
4,14010832.0,Science Fiction,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
5,14010832.0,Action,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
6,14010832.0,Adventure,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
7,NaN,Erotic thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye
8,NaN,Psychological thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye
9,NaN,Thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye


In [3]:
# drama = movies_split[movies_split['genres'] == 'Drama']
# comedy = movies_split[movies_split['genres'] == 'Comedy']
# action_adv = movies_split[movies_split['genres'] == 'Action/Adventure']
# romance = movies_split[movies_split['genres'] == 'Romance Film']
# thriller = movies_split[movies_split['genres'] == 'Thriller']
# romance.head(10)

top = movies_split[(movies_split['genres'] == 'Drama') | (movies_split['genres'] == 'Comedy') |
                  (movies_split['genres'] == 'Action/Adventure') | (movies_split['genres'] == 'Romance Film') |
                  (movies_split['genres'] == 'Thriller')]
top.head(5)

,box_office_revenue,genres,release_date,runtime,summary,titles
3,14010832.0,Thriller,2001-08-24,98.0,"Set in the second half of the 22nd century, th...",Ghosts of Mars
9,NaN,Thriller,1987,110.0,A series of murders of rich young women throug...,White Of The Eye
10,NaN,Drama,1983,106.0,"Eva, an upper class housewife, becomes frustra...",A Woman in Flames
15,NaN,Romance Film,1997-04-04,93.0,"Adam, a San Francisco-based artist who works a...",Little city
18,NaN,Comedy,1997-04-04,93.0,"Adam, a San Francisco-based artist who works a...",Little city


In [5]:
sample = romance['summary'][15]
sample

NameError: name 'romance' is not defined

In [4]:
def tokenizer(blob):
    translator = str.maketrans({key: None for key in string.punctuation})
    
    tokenized = blob.lower().replace("-"," ").translate(translator).split()
    return tokenized

# tokens = tokenizer(sample)

In [5]:
summary_dict = {}
vocab = {}

for title, summary in zip(top['titles'], top['summary']):
    summary_dict[title] = tokenizer(summary)
    for token in tokenizer(summary):
        vocab[token] = vocab.get(token, 0) + 1

word_index = {}
for idx, word in enumerate(sorted(vocab)): 
        word_index[word] = idx

In [7]:
movie_vec = {title: np.zeros(len(word_index)) for title in top['titles']}

for title, summary in summary_dict.items():
    for word in summary:
        movie_vec[title][word_index[word]] += 1
movie_vec

MemoryError: 